In [1]:
import pymysql
import pandas as pd
import time
import numpy as np
import random
import os
import sys
import codecs
import jieba
import pkuseg
import math
import pickle
import gensim
import stanfordnlp as stanfordnlp
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.models.word2vec import LineSentence
from sklearn.manifold import TSNE
import re, sys, codecs
import hanziconv
from hanziconv import HanziConv
import pytest
import re
import nltk
from nltk.tree import Tree
import matplotlib.pyplot as plt
%matplotlib inline

from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
from nltk.tokenize.stanford_segmenter import StanfordSegmenter
import stanfordcorenlp
import stanfordnlp
from stanfordcorenlp import StanfordCoreNLP

from functools import wraps
from collections import defaultdict
from collections import Counter
from functools import lru_cache

### 数据库读取数据（但数据存在截断，未使用其数据）

In [2]:
file_name = "/Users/lingrowzhang/Documents/GitHub/Artificial-Intelligence-for-NLP/"

In [3]:
headname = ["id","author","source","content","feature","title","url"]
news_chinese = pd.read_csv(file_name + "date/news_chinese.csv")
news_chinese.columns = headname

In [4]:
news_chinese.head(2)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm


In [5]:
data_content = list(news_chinese["content"])
data_content[6]

'虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是今年 3 月份官方突然宣布回归中国，预示着很快就有夏普新机在中国登场了。那么，第一款夏普手机什么时候登陆中国呢？又会是怎么样的手机呢？\r\n近日，一款型号为 FS8016 的夏普神秘新机悄然出现在 GeekBench 的跑分库上。从其中相关信息了解到，这款机子并非旗舰定位，所搭载的是高通骁龙 660 处理器，配备有 4GB 的内存。骁龙 660 是高通今年最受瞩目的芯片之一，采用 14 纳米工艺，八个 Kryo 260 核心设计，集成 Adreno 512 GPU 和 X12 LTE 调制解调器。\r\n当前市面上只有一款机子采用了骁龙 660 处理器，那就是已经上市销售的 OPPO R11。骁龙 660 尽管并非旗舰芯片，但在多核新能上比去年骁龙 820 强，单核改进也很明显，所以放在今年仍可以让很多手机变成高端机。不过，由于 OPPO 与高通签署了排他性协议，可以独占两三个月时间。\r\n考虑到夏普既然开始测试新机了，说明只要等独占时期一过，夏普就能发布骁龙 660 新品了。按照之前被曝光的渲染图了解，夏普的新机核心竞争优势还是全面屏，因为从 2013 年推出全球首款全面屏手机 EDGEST 302SH 至今，夏普手机推出了多达 28 款的全面屏手机。\r\n在 5 月份的媒体沟通会上，惠普罗忠生表示：“我敢打赌，12 个月之后，在座的各位手机都会换掉。因为全面屏时代的到来，我们怀揣的手机都将成为传统手机。”\r\n'

In [10]:
%%time
jieba.cut(data_content[6])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


<generator object Tokenizer.cut at 0x1a33969410>

In [9]:
%%time
list(jieba.cut(data_content[6]))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/04/ww1l0rx91jjf64l2y45_44d80000gn/T/jieba.cache
Loading model cost 0.697 seconds.
Prefix dict has been built succesfully.


CPU times: user 640 ms, sys: 35.7 ms, total: 676 ms
Wall time: 704 ms


['虽然',
 '至今',
 '夏普',
 '智能手机',
 '在',
 '市场',
 '上',
 '无法',
 '排得',
 '上',
 '号',
 '，',
 '已经',
 '完全',
 '没落',
 '，',
 '并',
 '于',
 ' ',
 '2013',
 ' ',
 '年',
 '退出',
 '中国',
 '市场',
 '，',
 '但是',
 '今年',
 ' ',
 '3',
 ' ',
 '月份',
 '官方',
 '突然',
 '宣布',
 '回归',
 '中国',
 '，',
 '预示',
 '着',
 '很快',
 '就',
 '有',
 '夏普',
 '新机',
 '在',
 '中国',
 '登场',
 '了',
 '。',
 '那么',
 '，',
 '第一款',
 '夏普',
 '手机',
 '什么',
 '时候',
 '登陆',
 '中国',
 '呢',
 '？',
 '又',
 '会',
 '是',
 '怎么样',
 '的',
 '手机',
 '呢',
 '？',
 '\r\n',
 '近日',
 '，',
 '一款',
 '型号',
 '为',
 ' ',
 'FS8016',
 ' ',
 '的',
 '夏普',
 '神秘',
 '新机',
 '悄然',
 '出现',
 '在',
 ' ',
 'GeekBench',
 ' ',
 '的',
 '跑',
 '分库',
 '上',
 '。',
 '从',
 '其中',
 '相关',
 '信息',
 '了解',
 '到',
 '，',
 '这',
 '款',
 '机子',
 '并非',
 '旗舰',
 '定位',
 '，',
 '所',
 '搭载',
 '的',
 '是',
 '高通',
 '骁龙',
 ' ',
 '660',
 ' ',
 '处理器',
 '，',
 '配备',
 '有',
 ' ',
 '4GB',
 ' ',
 '的',
 '内存',
 '。',
 '骁龙',
 ' ',
 '660',
 ' ',
 '是',
 '高通',
 '今年',
 '最受',
 '瞩目',
 '的',
 '芯片',
 '之一',
 '，',
 '采用',
 ' ',
 '14',
 ' ',
 '纳米',
 '工艺',
 '，',
 '八个',
 ' ',
 'Kryo',
 

In [7]:
%%time
pkuseg.pkuseg().cut(data_content[6])

CPU times: user 2.09 s, sys: 447 ms, total: 2.54 s
Wall time: 2.62 s


['虽然',
 '至今',
 '夏普',
 '智能',
 '手机',
 '在',
 '市场',
 '上',
 '无法',
 '排',
 '得',
 '上',
 '号',
 '，',
 '已经',
 '完全',
 '没落',
 '，',
 '并',
 '于',
 '2013',
 '年',
 '退出',
 '中国',
 '市场',
 '，',
 '但是',
 '今年',
 '3',
 '月份',
 '官方',
 '突然',
 '宣布',
 '回归',
 '中国',
 '，',
 '预示',
 '着',
 '很快',
 '就',
 '有',
 '夏普',
 '新机',
 '在',
 '中国',
 '登场',
 '了',
 '。',
 '那么',
 '，',
 '第一',
 '款',
 '夏普',
 '手机',
 '什么',
 '时候',
 '登陆',
 '中国',
 '呢',
 '？',
 '又',
 '会',
 '是',
 '怎么样',
 '的',
 '手机',
 '呢',
 '？',
 '近日',
 '，',
 '一',
 '款',
 '型号',
 '为',
 'FS8016',
 '的',
 '夏普',
 '神秘',
 '新机',
 '悄然',
 '出现',
 '在',
 'GeekBench',
 '的',
 '跑分库',
 '上',
 '。',
 '从',
 '其中',
 '相关',
 '信息',
 '了解',
 '到',
 '，',
 '这',
 '款',
 '机子',
 '并非',
 '旗舰',
 '定位',
 '，',
 '所',
 '搭载',
 '的',
 '是',
 '高通',
 '骁龙',
 '660',
 '处理器',
 '，',
 '配备',
 '有',
 '4GB',
 '的',
 '内存',
 '。',
 '骁龙',
 '660',
 '是',
 '高通',
 '今年',
 '最',
 '受',
 '瞩目',
 '的',
 '芯片',
 '之一',
 '，',
 '采用',
 '14',
 '纳米',
 '工艺',
 '，',
 '八',
 '个',
 'Kryo',
 '260',
 '核心',
 '设计',
 '，',
 '集成',
 'Adreno',
 '512',
 'GPU',
 '和',
 'X12',
 'LTE',
 '调

In [11]:
# 提取其中有用数据
all_author = []
all_type = []

In [12]:
all_source = []
for the_tmp in list(news_chinese["source"]):
    for the_str in str(the_tmp).split('@'):
        if "http" not in the_str and the_str != "":
            if "第" in the_str:
                try:
                    the_str_i = the_str.index("第")
                except:
                    continue
                if the_str[:the_str_i] not in all_source:
                    all_source.append(the_str[:the_str_i])
            else:
                if the_str not in all_source:
                    all_source.append(the_str)
all_source    

['快科技',
 '新华社',
 '深圳大件事',
 '中国证券报?中证网',
 '威锋网',
 '荆楚网-楚天都市报',
 '证券时报网',
 '中国新闻网',
 '国际在线',
 '中国证券网',
 'solidot',
 '新华网',
 '泡泡网',
 '凤凰体育',
 '雷锋网',
 '中超球评',
 '中国经济周刊',
 '人民日报',
 '央广网',
 '体育君',
 '新浪体育',
 '广州日报',
 'TechWeb',
 '足球小志',
 '人民网',
 '时代财经',
 '新疆日报',
 '中关村在线',
 '网易科技',
 '封面新闻',
 '央视新闻',
 '澎湃新闻网',
 '中青在线',
 '央视新闻客户端',
 '千龙网',
 '新浪综合',
 '中新网',
 'GamerSky游民星空',
 '中国经济网',
 'cnBeta.COM',
 '热点体育',
 '枪炮篮球PiU',
 '中国网财经',
 '知客数码',
 '于斌',
 '霍寿喜',
 '网易科学人',
 '看看新闻KNEWS',
 '最后一公里',
 '伊吾零距离',
 '环球时报-环球网',
 '经济日报',
 '足球部落国际版',
 '参考消息网',
 '黑龙江日报',
 '解放军报',
 '环球网',
 '星城古龙',
 '凤凰网科技',
 '每日经济新闻',
 '新浪科技',
 '经济参考报',
 '三十年莱斯特城球',
 '中国科技网',
 '柳州新闻网',
 '新京报即时新闻',
 'CCTV-央视网',
 '新闻晨报',
 '北青网-北京青年报',
 'linkgou.com',
 '中华网投资',
 '环球科技',
 '北京旅游网',
 '检察日报',
 '海南日报',
 '南方周末',
 '中国台湾网',
 '南宁晚报',
 '时刻体育',
 '北京晨报',
 '凤凰娱乐',
 '球娱天下',
 '中国娱乐网',
 '云南网',
 '中国时尚在线',
 '足球喜剧之王',
 '南国都市报',
 '剥洋葱people',
 '安卓中国',
 '中国青年报',
 '中国网',
 '网络',
 '深读',
 '哎，听说',
 '阳郊网警',
 '燕赵晚报',
 '证券日报',
 '重案组37号',
 '郑州晚报',
 '华尔街见闻',
 'Wind资讯',


In [13]:
all_author = []
for the_tmp in list(news_chinese["source"]):
    the_str = str(the_tmp)
    if "http" not in the_str and the_str != "" and the_str not in all_source:
            all_source.append(the_str)

## 模型前的数据处理

In [14]:
def tradition2simple(line):
    # 将繁体转换成简体
    return HanziConv.toSimplified(line)
# pkuseg分词
def pkuseg_cut(sent):
    seg = pkuseg.pkuseg()
    return seg.cut(sent)
def cut(string):
    return list(jieba.cut(string))

In [15]:
# 读入停用词表
stop_symbols = pd.read_csv(file_name+"date/test/stop_symbols.csv", encoding="utf-8")

In [16]:
def get_word(string_list):
    the_str  = ""
    for string in string_list:
        for the_chat in string:
            if ('\u4e00' <= the_chat <= '\u9fa5'):
                the_str += the_chat
        if the_str == "" or the_str == " ":
            
        the_str = the_str + " "
    return the_str
all_word = defaultdict(int)
def del_stop_word(string):
    the_string = ""
    for the_word in string.split(" "):
        if the_word in stop_symbols or the_word == "":
            continue
        else:
            all_word[the_word] += 1
            the_string += the_word + " "
    return the_string

In [18]:
def wiki_words(file_index):
    with open(file_name + "date/wiki_66/deal_" + str(file_index) + '.txt', 'r', encoding='UTF-8') as f, \
            open(file_name + "date/wiki_66/pkuseg_true_" + str(file_index) + '.txt', 'w', encoding='UTF-8') as pkuseg_path:
        lines = [''.join(line) for line in f.readlines() if
                 not line.startswith("<doc") and not line.startswith("</doc>") and not len(line.strip()) == 0]
        for i, line in enumerate(lines):
            get_word_str = get_word(cut(line))
            pkuseg_path.write(get_word_str + '\n')
            # del_stop_word_str = del_stop_word(get_word_str)
    return True

In [19]:
for path,_,files in os.walk(file_name+ "date/wiki_66/AA/"):
    for the_filename in files:
        if the_filename ==  '.DS_Store':
            continue
        wiki_words(the_filename[len(the_filename)-2:])
        print(the_filename)

KeyboardInterrupt: 

def try_readlines(file_index):
    with open(file_name + "date/wiki_66/AA/wiki_"+ str(file_index), 'r', encoding='UTF-8') as f, \
            open(file_name + "date/wiki_66/deal_" + str(file_index) + '.txt', 'w', encoding='UTF-8') as s, \
            open(file_name + "date/wiki_66/pkuseg_" + str(file_index) + '.txt', 'w', encoding='UTF-8') as pkuseg_path:
        lines = [''.join(line) for line in f.readlines() if
                 not line.startswith("<doc") and not line.startswith("</doc>") and not len(line.strip()) == 0]
        for i, line in enumerate(lines):
            line_simple = tradition2simple(line.strip('\n'))
            s.write(line_simple + '\n')
            get_word_str = ''.join(pkuseg.pkuseg().cut(line_simple))
            # del_stop_word_str = del_stop_word(get_word_str)
            pkuseg_path.write(get_word_str + '\n')

for path,_,files in os.walk(file_name+ "date/wiki_66/AA/"):
    for the_filename in files:
        if the_filename ==  '.DS_Store':
            continue
        star_time = time.clock()
        try_readlines(the_filename[len(the_filename)-2:])
        print(the_filename)

## 训练一个模型wiki

In [ ]:
i=0
for path,_,files in os.walk(file_name + "date/wiki_66/"):
    for the_file in files:
        if "wiki" in str(the_file):
            with open(file_name + 'date/wiki_66/pkuseg_' + the_file[len(the_file)-2:] + '.txt', 'r', encoding='utf-8') as wiki_news:
                sentences = LineSentence(wiki_news)
                if i == 0:
                    i+=1
                    model_wiki_200 = Word2Vec(sentences, size=200, min_count=2, window=6, workers=8)
                else:
                    model_wiki_200.build_vocab(sentences, update=True)  # 更新词汇表
                    model_wiki_200.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # epoch=iter语料库的迭代次数；（默认为5）  total_examples:句子数。
                print(the_file)

wiki_11


/Users/lingrowzhang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if sys.path[0] == '':


wiki_16


In [ ]:
model_wiki_200.save('word2vec_wiki_200' + '.model')

In [ ]:
i=0
for path,_,files in os.walk(file_name + "date/wiki_66/"):
    for the_file in files:
        if "wiki" in str(the_file):
            with open(file_name + 'date/wiki_66/pkuseg_' + the_file[len(the_file)-2:] + '.txt', 'r', encoding='utf-8') as wiki_news:
                sentences = LineSentence(wiki_news)
                if i == 0:
                    i+=1
                    model_wiki_500 = Word2Vec(sentences, size=500, min_count=2, window=7, workers=8)
                else:
                    model_wiki_500.build_vocab(sentences, update=True)  # 更新词汇表
                    model_wiki_500.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # epoch=iter语料库的迭代次数；（默认为5）  total_examples:句子数。
                print(the_file)

In [ ]:
model_wiki_500.save('word2vec_wiki_500' + '.model')

## 训练一个wiki+新闻的模型

In [ ]:
content = []
for the_content in data_content:
    content += tradition2simple(str(the_content).strip('\n')).split('\r\n')
data_content = []
for the_content in content:
    data_content.append(' '.join(cut(the_content)))
tmp_data = pd.DataFrame(data = data_content)

In [ ]:
tmp_data.to_csv(file_name + "date/content.csv", encoding='utf-8')

In [ ]:
model_new = Word2Vec.load(file_name +'model/word2vec_500.model')
with open(file_name + "date/content.csv", 'r', encoding='utf-8') as wiki_news:
    sentences = LineSentence(wiki_news)
    model_wiki_new_500.build_vocab(sentences, update=True)  # 更新词汇表
    model_wiki_new_500.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # epoch=iter语料库的迭代次数；（默认为5）  total_examples:句子数。
    model_wiki_new_500.save(file_name + 'model/word2vec_wiki_new_500'+'.model')

In [ ]:
model_new = Word2Vec.load(file_name +'model/word2vec_200.model')
with open(file_name + "date/content.csv", 'r', encoding='utf-8') as wiki_news:
    sentences = LineSentence(wiki_news)
    model_wiki_new_200.build_vocab(sentences, update=True)  # 更新词汇表
    model_wiki_new_200.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # epoch=iter语料库的迭代次数；（默认为5）  total_examples:句子数。
    model_wiki_new_200.save(file_name + 'model/word2vec_wiki_new_200'+'.model')

## 新闻模型

In [ ]:
with open(file_name + "date/content.csv", 'r', encoding='utf-8') as wiki_news:
    sentences = LineSentence(wiki_news)
    model_the_new_200 = Word2Vec(sentences, size=500, min_count=2, window=7, workers=8)
    model_the_new_200.save(file_name + 'model/word2vec_news500'+'.model')

In [ ]:
with open(file_name + "date/content.csv", 'r', encoding='utf-8') as wiki_news:
    sentences = LineSentence(wiki_news)
    model_the_new_500 = Word2Vec(sentences, size=200, min_count=2, window=7, workers=8)
    model_the_new_500.save(file_name + 'model/word2vec_news500'+'.model')

## 输出同义词

In [ ]:
word = "说"

In [ ]:
model.wv.most_similar(word, topn=200)

In [ ]:
model.wv.most_similar(word, topn=200)

In [ ]:
model.wv.most_similar(word, topn=200)

In [ ]:
model_new.wv.most_similar(word, topn=200)

In [ ]:
# 找相似的词画图
def tsne_plot(model):
    labels = []
    tokens = []
    
    words = ['对话','理解','说话','沟通']
    for word in words:
        labels.append(word)
        tokens.append(model[word])
        for sim, _ in model.wv.most_similar(word, topn=100):
            labels.append(sim)
            tokens.append(model[sim])
        
    tsne = TSNE(perplexity=40, init='pca', n_iter=3500, random_state=42)
    new_vectors = tsne.fit_transform(tokens)
    
    x, y = [], []
    for v1, v2 in new_vectors:
        x.append(v1)
        y.append(v2)
        
    plt.figure(figsize=(16, 16))
    for i in range(len(x)):
        plt.scatter(x[i], y[i])
        plt.annotate(labels[i], xy=(x[i], y[i]), xytext=(3, 2), textcoords='offset points', ha='center', va='bottom')
        
    plt.show()
tsne_plot(model)

### 对文章进行句法分析，观察规律（找10句话）

In [9]:
zh_model = StanfordCoreNLP(file_name+r"project_1/stanford-corenlp-full-2018-10-05", port=999, lang='zh')

In [10]:
the_content = data_content[0]
the_content

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n'

In [11]:
print('Tokenize:', zh_model.word_tokenize(the_content))

Tokenize: ['此外', '，', '自', '本周', '（', '6月', '12日', '）', '起', '，', '除', '小米', '手机', '6', '等', '15', '款', '机型', '外', '，', '其余', '机型', '已', '暂停', '更新', '发布', '（', '含', '开发', '版', '/', '体验', '版', '内测', '，', '稳', '定版', '暂', '不', '受', '影响', '）', '，', '以', '确保', '工程师', '可以', '集中', '全部', '精力', '进行', '系统', '优化', '工作', '。', '有', '人', '猜测', '这', '也', '是', '将', '精力', '主要', '用到', 'MIUI', '9', '的', '研发', '之中', '。', 'MIUI', '8', '去年', '5月', '发布', '，', '距今', '已', '有', '一', '年', '有余', '，', '也', '是', '时候', '更新', '换代', '了', '。', '当然', '，', '关于', 'MIUI', '9', '的', '确切', '信息', '，', '我们', '还是', '等待', '官方', '消息', '。']


In [12]:
print('Part of Speech:', zh_model.pos_tag(the_content))

Part of Speech: [('此外', 'AD'), ('，', 'PU'), ('自', 'P'), ('本周', 'NT'), ('（', 'PU'), ('6月', 'NT'), ('12日', 'NT'), ('）', 'PU'), ('起', 'VV'), ('，', 'PU'), ('除', 'P'), ('小米', 'NN'), ('手机', 'NN'), ('6', 'CD'), ('等', 'M'), ('15', 'CD'), ('款', 'M'), ('机型', 'NN'), ('外', 'LC'), ('，', 'PU'), ('其余', 'DT'), ('机型', 'NN'), ('已', 'AD'), ('暂停', 'VV'), ('更新', 'NN'), ('发布', 'VV'), ('（', 'PU'), ('含', 'VV'), ('开发', 'VV'), ('版', 'NN'), ('/', 'PU'), ('体验', 'NN'), ('版', 'NN'), ('内测', 'NN'), ('，', 'PU'), ('稳', 'AD'), ('定版', 'VV'), ('暂', 'AD'), ('不', 'AD'), ('受', 'VV'), ('影响', 'NN'), ('）', 'PU'), ('，', 'PU'), ('以', 'MSP'), ('确保', 'VV'), ('工程师', 'NN'), ('可以', 'VV'), ('集中', 'VV'), ('全部', 'DT'), ('精力', 'NN'), ('进行', 'VV'), ('系统', 'NN'), ('优化', 'NN'), ('工作', 'NN'), ('。', 'PU'), ('有', 'VE'), ('人', 'NN'), ('猜测', 'VV'), ('这', 'PN'), ('也', 'AD'), ('是', 'VC'), ('将', 'BA'), ('精力', 'NN'), ('主要', 'AD'), ('用到', 'VV'), ('MIUI', 'NR'), ('9', 'CD'), ('的', 'DEG'), ('研发', 'NN'), ('之中', 'LC'), ('。', 'PU'), ('MIUI', 'NR'), ('8', '

In [13]:
# 句法成分分析
print('Constituency Parsing:', zh_model.parse(the_content) + "\n")

Constituency Parsing: (ROOT
  (IP
    (VP
      (IP
        (IP
          (ADVP (AD 此外))
          (PU ，)
          (PP (P 自)
            (NP (NT 本周) (PU （) (NT 6月) (NT 12日) (PU ）) (VV 起)))
          (PU ，)
          (PP (P 除)
            (LCP
              (NP
                (NP (NN 小米) (NN 手机))
                (QP (CD 6)
                  (CLP (M 等)))
                (QP (CD 15)
                  (CLP (M 款)))
                (NP (NN 机型)))
              (LC 外)))
          (PU ，)
          (NP
            (DP (DT 其余))
            (NP (NN 机型)))
          (VP
            (ADVP (AD 已))
            (VP (VV 暂停)
              (IP
                (NP (NN 更新))
                (VP (VV 发布))))
            (PRN (PU （)
              (VP (VV 含)
                (NP (VV 开发) (NN 版)))
              (PU /)
              (NP
                (NP (NN 体验) (NN 版))
                (NP (NN 内测))))))
        (PU ，)
        (IP
          (VP
            (ADVP (AD 稳))
            (VP (VV 定版)
              (IP
    

In [14]:
# 依存句法分析
print('Dependency:', zh_model.dependency_parse(the_content))

Dependency: [('ROOT', 0, 24), ('advmod', 24, 1), ('punct', 24, 2), ('case', 9, 3), ('nmod:tmod', 9, 4), ('punct', 7, 5), ('compound:nn', 7, 6), ('parataxis:prnmod', 4, 7), ('punct', 7, 8), ('nmod:prep', 24, 9), ('punct', 24, 10), ('case', 18, 11), ('compound:nn', 13, 12), ('compound:nn', 18, 13), ('nummod', 18, 14), ('mark:clf', 14, 15), ('nummod', 18, 16), ('mark:clf', 16, 17), ('nmod:prep', 24, 18), ('case', 18, 19), ('punct', 24, 20), ('det', 22, 21), ('nsubj', 24, 22), ('advmod', 24, 23), ('dobj', 24, 25), ('conj', 24, 26), ('punct', 28, 27), ('dep', 26, 28), ('compound:nn', 30, 29), ('dobj', 28, 30), ('punct', 28, 31), ('compound:nn', 33, 32), ('compound:nn', 34, 33), ('dep', 28, 34), ('punct', 28, 35), ('advmod', 37, 36), ('conj', 28, 37), ('advmod', 40, 38), ('neg', 40, 39), ('dep', 37, 40), ('dobj', 40, 41), ('punct', 40, 42), ('punct', 28, 43), ('aux:prtmod', 45, 44), ('conj', 28, 45), ('nsubj', 48, 46), ('aux:modal', 48, 47), ('ccomp', 45, 48), ('det', 50, 49), ('dobj', 48, 5

In [15]:
the_content_j = the_content.split("。")
the_content_j

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作',
 '有人猜测这也是将精力主要用到MIUI 9的研发之中',
 '\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了',
 '\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息',
 '\r\n']

In [ ]:
tree = Tree.fromstring(zh_model.parse(the_content))
tree.draw()
zh_model.close()

## 使用哈工大的ltp进行分析

In [ ]:
/Users/lingrowzhang/Documents/GitHub/Artificial-Intelligence-for-NLP/ltp_data_v3.4.0